In [35]:
import pandas as pd

In [36]:
df = pd.read_csv("nu.csv")
df_bb = pd.read_csv("bb.csv")
df_cartao = pd.read_csv("cartao.csv")

In [37]:
df.head()

,Data,Valor,Identificador,Descrição
0,01/03/2025,-33.59,67c31440-02e2-43c9-93a5-067fd6cdec92,Compra no débito - Tenda Atacado S A
1,01/03/2025,-14.98,67c316c6-9427-4d46-b69b-aa8d564d7dce,Compra no débito - Lojas Americanas
2,01/03/2025,-4.00,67c31756-f8cc-4027-85cd-b030fcfbb16b,Compra no débito - Buriti Shopping Guara
3,02/03/2025,-51.80,67c46229-5907-4dd2-b38d-1bf77251dc30,Compra no débito - Br+Mais Conveniencia
4,02/03/2025,-39.76,67c4641e-69ca-442b-982d-f3f5a3661c32,Compra no débito - Colinas Supermercado


In [38]:
df.drop("Identificador", axis=1, inplace=True)

In [39]:
# date	description	type	amount
df.rename(
  columns={"Data": "date","Descrição": "description","Valor": "amount"}, 
  inplace=True
)

In [40]:
df_bb.rename(columns={
  'Data': 'date',
  'Valor': 'amount'
}, inplace=True)

df_bb['amount'] = df_bb['amount'].replace({r'"': '', r'\.': '', r',': '.'}, regex=True).astype(float)

df_bb['description'] = df_bb['Lancamento'] + ' - ' + df_bb['Detalhes']

df_bb = df_bb[['date', 'description', 'amount']]

In [41]:
df_cartao.rename(columns = { 'title': 'description' }, inplace = True)
df_cartao['amount'] = df_cartao['amount'].astype('float') * -1
df_cartao = df_cartao[['date', 'description', 'amount']]

In [42]:
df = pd.concat([df, df_bb, df_cartao], ignore_index=True)

In [43]:
df['source'] = pd.NA
df['categories'] = pd.NA

In [44]:
df["type"] = df["amount"].map(lambda value: "INCOME" if value > 0 else "OUTCOME")

In [45]:
df.head()

,date,amount,description,source,categories,type
0,01/03/2025,-33.59,Compra no débito - Tenda Atacado S A,<NA>,<NA>,OUTCOME
1,01/03/2025,-14.98,Compra no débito - Lojas Americanas,<NA>,<NA>,OUTCOME
2,01/03/2025,-4.00,Compra no débito - Buriti Shopping Guara,<NA>,<NA>,OUTCOME
3,02/03/2025,-51.80,Compra no débito - Br+Mais Conveniencia,<NA>,<NA>,OUTCOME
4,02/03/2025,-39.76,Compra no débito - Colinas Supermercado,<NA>,<NA>,OUTCOME


In [46]:
mapperDf = pd.read_csv("mapper.tsv", sep="\t")
mapper = mapperDf.to_records().tolist()

In [47]:
def mapRow(row):
  for mapp in mapper:
    description = str(row["description"]).lower()
    amount = int(row['amount'])
    
    findKey = str(mapp[1]).lower()
    mapAmount = int(mapp[2])
    mapDescription = str(mapp[3])
    source = str(mapp[4])
    categories = str(mapp[5])

    if findKey not in description:
      continue
    
    if mapAmount != 0 and mapAmount != amount:
      continue

    row['description'] = mapDescription
    row['source'] = source
    row['categories'] = categories
    break

  if row['source'] == None or pd.isna(row['source']) or row['source'] == "":
    row['description'] = description.replace("transferência enviada pelo ", "").replace("compra no ", "")
    row['source'] = "CONTA CORRENTE"
    row['categories'] = "OUTROS"

  return row 

In [48]:
df = df.apply(mapRow, axis=1)
df.head()

,date,amount,description,source,categories,type
0,01/03/2025,-33.59,débito - tenda atacado s a,CONTA CORRENTE,OUTROS,OUTCOME
1,01/03/2025,-14.98,débito - lojas americanas,CONTA CORRENTE,OUTROS,OUTCOME
2,01/03/2025,-4.00,débito - buriti shopping guara,CONTA CORRENTE,OUTROS,OUTCOME
3,02/03/2025,-51.80,Padaria,CONTA CORRENTE,ALIMENTAÇÃO,OUTCOME
4,02/03/2025,-39.76,Compras Supermercado,CONTA CORRENTE,ALIMENTAÇÃO,OUTCOME


In [49]:
df = df[~df['description'].isin(["nan", "EXCLUIR"])]

In [50]:
df.to_csv(
  "./nu.tsv", 
  sep="\t", 
  lineterminator='\r\n', 
  na_rep="", 
  index=False,
  columns=["date", "description", "type", "amount", "source", "categories"],
  decimal=","
)